# Human Like Quesitons

In [1]:
import os
from openai import OpenAI
import pandas as pd
import json

openai_api_key = os.environ.get("OPENAI_API_KEY")
client = OpenAI()

#### Load Existing Questons

In [2]:
# from src.gdrive_api.utils import extract_questions, find_and_load_all_problems, extract_questions_by_topic

# parent_folder_path = './notebooks'

# all_problems = find_and_load_all_problems(parent_folder_path)
# questions = extract_questions(all_problems)
# existing_questions = list(questions)
# existing_questions

In [2]:
from src.gdrive_api.utils import extract_questions, find_and_load_all_problems, extract_questions_by_topic

parent_dir = "./notebooks"
all_problems = find_and_load_all_problems(parent_dir)
questions_grouped_by_topic = extract_questions_by_topic(all_problems)

# # Printing each topic and its questions
# for topic, questions in questions_grouped_by_topic.items():
#     print(f"Topic: {topic}")
#     for question in questions:
#         print(f" - {question}")

TypeError: unhashable type: 'dict'

In [4]:
len(all_problems)

2676

In [8]:

questions_by_topic = extract_questions_by_topic(all_problems)

#### Load Topic

In [3]:
import json

# Function to crawl the topic hierarchy and retrieve all topics
def crawl_keys(d, sep=' > ', prefix=''):
    paths = []
    for k, v in d.items():
        path = prefix + k
        if isinstance(v, dict) and len(v.keys()) > 0:
            paths.extend(crawl_keys(v, sep, path + sep))
        else:
            paths.append(path)
    return paths

# Open the `topic_hierarchy.json` file and retrieve all topics
with open('topic_hierarchy.json') as json_file:
    topic_hierarchy = json.load(json_file)

all_topics = crawl_keys(topic_hierarchy)
print(f"Total number of topics: {len(all_topics)}")

Total number of topics: 132


#### Max Questions to generate

In [95]:
MAX_QUESTIONS = 250  # The maximum number of questions
generated_questions_count = 0  # Counter for the number of questions generated so far

In [ ]:
from src.gdrive_api.utils import generate_human_like_questions
from src.gdrive_api.utils import extract_questions, find_and_load_all_problems, extract_questions_by_topic
from tqdm import tqdm

problems = []

questions_grouped_by_topics = extract_questions_by_topic(all_problems)

# If the existing questions are not grouped by full topic paths, convert them first
questions_grouped_by_topic = {}
for topic in all_topics:
    questions_grouped_by_topic[topic] = questions_grouped_by_topics.get(topic, set())

with tqdm(total=MAX_QUESTIONS) as pbar:
    for topic, existing_questions in questions_grouped_by_topics.items():
        # Stop if we've reached the max limit
        if generated_questions_count >= MAX_QUESTIONS:
            break

        # Generate questions
        questions = generate_human_like_questions(topic, 1, existing_questions)
        for question in questions["questions"]:
            # If we're at the max, break
            if generated_questions_count >= MAX_QUESTIONS:
                break
            problems.append({
                "metadata": {
                    "topic": topic,
                    "type": "query",
                    "difficulty": "Easy",
                    "target_length": 1
                },
                "messages": [
                    {"role": "user", "content": question},
                ]
            })
            generated_questions_count += 1  # Increment for each question
            pbar.update(1)

In [96]:
len(existing_questions)

8

In [97]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from src.gdrive_api.utils import generate_human_like_questions
from tqdm import tqdm

questions_grouped_by_topics = extract_questions_by_topic(all_problems)
questions_grouped_by_topic = {}

# Convert if the existing questions are not grouped by full topic paths
for topic in all_topics:
    questions_grouped_by_topic[topic] = questions_grouped_by_topics.get(topic, set())

problems = []
generated_questions_count = 0 

# Function to generate questions for a given topic
def generate_for_topic(topic):
    global generated_questions_count  # To allow modification of the global counter

    new_problems = []
    existing_questions = questions_grouped_by_topic[topic]

    if generated_questions_count < MAX_QUESTIONS:
        questions = generate_human_like_questions(topic, 3, existing_questions)
        for question in questions["questions"]:
            if generated_questions_count >= MAX_QUESTIONS:
                break
            new_problems.append({
                "metadata": {
                    "topic": topic,
                    "type": "query",
                    "difficulty": "Easy",
                    "target_length": 1
                },
                "messages": [
                    {"role": "user", "content": question},
                ]
            })
            generated_questions_count += 1
    return new_problems

# Set the maximum number of threads to use
MAX_THREADS = 20

# Use ThreadPoolExecutor to run the tasks asynchronously
with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    # Create a dictionary to store future-to-topic mapping
    future_to_topic = {executor.submit(generate_for_topic, topic): topic for topic in questions_grouped_by_topic.keys()}

    with tqdm(total=MAX_QUESTIONS) as pbar:
        for future in as_completed(future_to_topic):
            new_problems = future.result()
            problems.extend(new_problems)
            # Update the progress bar by the number of new problems generated
            pbar.update(len(new_problems))

100%|██████████| 250/250 [00:39<00:00,  6.29it/s]


In [85]:
len(problems)

500

In [14]:

# from src.gdrive_api.utils import generate_human_like_questions
# from tqdm import tqdm

# problems = []

# with tqdm(total=MAX_QUESTIONS) as pbar:
#     for topic in all_topics:
#         # Stop if we've reached the max limit
#         if generated_questions_count >= MAX_QUESTIONS:
#             break

#         existing_questions = questions_grouped_by_topic.get(f'{topic}', set())
#         print('existing' + str(existing_questions))
#         print('topics' + str(topic))

#         # Generate questions
#         questions = generate_human_like_questions(topic, 5, existing_questions)
#         for question in questions["questions"]:
#             # If we're at the max, break
#             if generated_questions_count >= MAX_QUESTIONS:
#                 break
#             problems.append({
#                 "metadata": {
#                     "topic": topic,
#                     "type": "query",
#                     "difficulty": "Easy",
#                     "target_length": 1
#                 },
#                 "messages": [
#                     {"role": "user", "content": question},
#                 ]
#             })
#             generated_questions_count += 1  # Increment for each question
#             pbar.update(1)

In [82]:
MAX_QUESTIONS = 250  # The maximum number of questions
generated_questions_count = 0  # Counter for the number of questions generated so far

In [98]:
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed
from src.gdrive_api.utils import generate_human_like_code_modification_requests
from tqdm import tqdm

# Initialize the lock and global variables
count_lock = threading.Lock()
generated_questions_count = 0  # Initialize this global counter

# Function to generate code modification requests for a given topic
def generate_requests_for_topic(topic_and_questions):
    global generated_questions_count  # Declare as global to modify the global variable
    topic, existing_questions = topic_and_questions
    new_problems = []

    with count_lock:
        can_generate_more = generated_questions_count < MAX_QUESTIONS

    if can_generate_more:
        questions = generate_human_like_code_modification_requests(topic, 4, existing_questions)
        
        for question in questions["questions"]:
            with count_lock:
                if generated_questions_count >= MAX_QUESTIONS:
                    break
                generated_questions_count += 1

            new_problems.append({
                "metadata": {
                    "topic": topic,
                    "type": "modification",
                    "difficulty": "Easy",
                    "target_length": 1
                },
                "messages": [
                    {"role": "user", "content": question},
                ]
            })

    return new_problems

# Maximum number of threads to use
MAX_THREADS = 20

with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    # Initialize tqdm outside of the threads
    with tqdm(total=MAX_QUESTIONS) as pbar:
        # Create and submit the tasks
        future_to_topic = {
            executor.submit(generate_requests_for_topic, (topic, existing_questions)): topic
            for topic, existing_questions in questions_grouped_by_topics.items()
        }

        # Gather the results as they are ready
        for future in as_completed(future_to_topic):
            new_problems = future.result()
            problems.extend(new_problems)

  0%|          | 0/250 [02:14<?, ?it/s]


In [99]:
problems

[{'metadata': {'topic': 'algorithms > by_topic > bit_manipulation',
   'type': 'query',
   'difficulty': 'Easy',
   'target_length': 1},
  'messages': [{'role': 'user',
    'content': 'Can you show me how to set the nth bit of a number to 1 in Python?'}]},
 {'metadata': {'topic': 'algorithms > by_topic > bit_manipulation',
   'type': 'query',
   'difficulty': 'Easy',
   'target_length': 1},
  'messages': [{'role': 'user',
    'content': 'How do I use bitwise operators to determine if all the bits of a number are set to 1?'}]},
 {'metadata': {'topic': 'algorithms > by_topic > bit_manipulation',
   'type': 'query',
   'difficulty': 'Easy',
   'target_length': 1},
  'messages': [{'role': 'user',
    'content': "What's the most efficient way to count trailing zeros in a binary representation of an integer in Python?"}]},
 {'metadata': {'topic': 'algorithms > by_topic > math',
   'type': 'query',
   'difficulty': 'Easy',
   'target_length': 1},
  'messages': [{'role': 'user',
    'content':

In [70]:
from src.gdrive_api.utils import generate_human_like_code_modification_requests

generated_questions_count = 0

with tqdm(total=MAX_QUESTIONS) as pbar:
    for topic, existing_questions in questions_grouped_by_topics.items():
        print(existing_questions)
        # Stop if we've reached the max limit
        if generated_questions_count >= MAX_QUESTIONS:
            break
        
        questions = generate_human_like_code_modification_requests(topic, 1, existing_questions)
        for question in questions["questions"]:
            # If we're at the max, break
            if generated_questions_count >= MAX_QUESTIONS:
                break
            problems.append({
                "metadata": {
                    "topic": topic,
                    "type": "modification",
                    "difficulty": "Easy",
                    "target_length": 1
                },
                "messages": [
                    {"role": "user", "content": question},
                ]
            })
            generated_questions_count += 1  # Increment for each question
            pbar.update(1)

  0%|          | 0/250 [00:00<?, ?it/s]

{"I'm trying to reverse an array in-place, but it's not working as expected. Here's my code snippet: arr = [10, 20, 30, 40, 50]. Can you modify it to correctly reverse the array without using additional space?", 'How can I reverse an array in Python without using built-in functions?', "Hey, I've got this array of integers and I need to find the sum. Could you tweak my code to do that? Here's what I have so far: nums = [1, 2, 3, 4, 5].", "I need to rotate an array to the right by k steps, but I'm stuck. Here's my current code: def rotate_array(arr, k): # ... Can you fill in the function to make it work?", "Can you provide a Python function to calculate the maximum subarray sum using Kadane's algorithm?", 'Could you show me a way to rotate the elements of an array to the right by k steps in Python?', 'I need to merge two sorted arrays into one sorted array in Python. How do I do that?', "What's the Python code to find the duplicate numbers in an array?"}


  0%|          | 1/250 [00:02<10:16,  2.48s/it]

{"I'm trying to detect a cycle in a linked list using Floyd's cycle-finding algorithm, but I'm not sure if I've implemented it correctly. Could you review the code and fix any issues? Here's what I have so far: ```def has_cycle(head): slow = head fast = head while fast and fast.next: slow = slow.next fast = fast.next.next if slow == fast: return True return False```", "I need to merge two sorted linked lists into one sorted linked list, but I want to optimize my current recursive solution to an iterative one. Here's my recursive approach: ```def merge_lists(l1, l2): if not l1: return l2 if not l2: return l1 if l1.val < l2.val: l1.next = merge_lists(l1.next, l2) return l1 else: l2.next = merge_lists(l1, l2.next) return l2``` Can you help me convert this to an iterative method?", 'Could you show me a function to detect a cycle in a linked list?', 'Can you help me write a Python class for a doubly linked list with basic operations?', "Hey, I've got this function for reversing a singly lin

  1%|          | 2/250 [00:06<14:28,  3.50s/it]

...

{'How would you write a recursive Python function to compute the nth Fibonacci number?', "I'm curious, how would you use recursion to perform a binary search on a sorted list in Python?", "I need to modify this recursive binary search function to return the index of the target if it's found, and -1 if it's not. Here's the current code: def binary_search(arr, low, high, target): if high >= low: mid = (high + low) // 2 if arr[mid] == target: return mid elif arr[mid] > target: return binary_search(arr, low, mid - 1, target) else: return binary_search(arr, mid + 1, high, target) else: return None", 'Can you show me how to implement a recursive function to calculate factorial in Python?', "What's the Python recursive solution for finding the greatest common divisor (GCD) of two numbers?", "I've got this recursive function for calculating factorial, but I'd like to add memoization to improve its efficiency. Can you help me modify it? Here's the code: def factorial(n): return 1 if n == 0 else

  6%|▌         | 14/250 [01:29<25:10,  6.40s/it]

{'How would I use Python to find an element in a rotated sorted array?', "I'm curious, how do you perform a breadth-first search in a graph using Python?", 'Could you provide an example of a Python function to search for a specific value in a binary search tree?', 'Can you show me how to implement a binary search in Python?', "Hey, I've got this binary search function, but it seems to be missing the condition to check if the target is the middle element. Can you add that for me? Here's the code: ```def binary_search(arr, target): low, high = 0, len(arr) - 1 while low <= high: mid = (low + high) // 2 if arr[mid] < target: low = mid + 1 elif arr[mid] > target: high = mid - 1 return -1```", "I wrote this code for a jump search, but I forgot to include the step where you check the final block linearly if the element isn't found in the jumped blocks. Can you insert that part? Here's the code: ```def jump_search(arr, x): length = len(arr) step = int(length**0.5) prev = 0 while arr[min(step, 

  6%|▌         | 14/250 [01:44<29:25,  7.48s/it]


KeyboardInterrupt: 

In [100]:
problem_titles = []
problem_topic_counts = {}
file_path_to_problem = {}
for problem in problems:
    topic_type = f'{problem["metadata"]["topic"].split(" > ")[-1]}__{problem["metadata"]["type"]}'
    idx = problem_topic_counts.get(topic_type, 0)
    title = f'{topic_type}__{idx}'

    modified_title = f'{title}_V3_A'

    problem_titles.append(modified_title)
    file_path_to_problem[f"{modified_title}.ipynb"] = problem
    problem_topic_counts[topic_type] = idx + 1

#### distribution for the number of turns

In [12]:
import nbformat
from nbformat.v4 import new_notebook, new_markdown_cell
import re
import os
import random

# Define distribution for the number of turns
distribution = {1: 25, 2: 50, 6: 25}

# Ensure that the distribution sums up to 100%
assert sum(distribution.values()) == 100, "The distribution must sum to 100%"

def weighted_choice(choices):
    total = sum(weight for choice, weight in choices)
    r = random.uniform(0, total)
    upto = 0
    for choice, weight in choices:
        if upto + weight >= r:
            return choice
        upto += weight

def select_number_of_turns(distribution):
    # Normalize distribution values to probabilities
    total = sum(distribution.values())
    normalized_distribution = {k: v/total for k, v in distribution.items()}
    turns = weighted_choice(normalized_distribution.items())
    return turns

for p, t in zip(problems, problem_titles):
    # Determine the parity based on the last digit in the title
    title_number = int(re.search(r'\d+$', t).group())  # Extract the last number from the title

    turns = select_number_of_turns(distribution)
    
    parity = title_number % 2

    # Create a new notebook
    notebook = new_notebook()

    # Add metadata
    metadata = f"""# Metadata

**Python Topics** - {p["metadata"]["topic"]}

**Type** - {p["metadata"]["type"]}

**Target Number of Turns (User + Assistant)** - {turns}
"""
    metadata_cell = new_markdown_cell(metadata)
    notebook.cells.append(metadata_cell)

    # Add conversation header
    conversation_header = "# Conversation"
    conversation_header_cell = new_markdown_cell(conversation_header)
    notebook.cells.append(conversation_header_cell)

    # Append conversation messages
    title = "**User**" if parity else "**Assistant**"
    for message in p["messages"]:
        msg_content = f"""{title}

{message["content"]}
"""
        conversation_message_cell = new_markdown_cell(msg_content)
        notebook.cells.append(conversation_message_cell)

    # Save the notebook
    notebook_path = f'notebooks/test/{t}.ipynb'
     
    try:
        os.makedirs(os.path.dirname(notebook_path))
    except FileExistsError:
        pass
     
    with open(notebook_path, 'w') as f:
        nbformat.write(notebook, f)

#### biased_normal_distribution

In [102]:
import json
import nbformat
import os
import re
import numpy as np
from nbformat.v4 import new_notebook, new_markdown_cell
from scipy.stats import rankdata

def flatten_complexity_scores(data, key_prefix='', scores_dict=None):
    if scores_dict is None:
        scores_dict = {}
    for key, value in data.items():
        if isinstance(value, dict):
            if 'complexity' in value:
                scores_dict[f'{key_prefix}.{key}'.strip('.')] = value['complexity']
            else:
                new_prefix = f'{key_prefix}.{key}'.strip('.')
                flatten_complexity_scores(value, new_prefix, scores_dict)
        elif isinstance(value, int):
            scores_dict[key_prefix] = value
    return scores_dict

with open('./topic_dist.json') as f:
    complexity_data = json.load(f)

# Flatten the complexity data into a dictionary
complexity_scores = flatten_complexity_scores(complexity_data)

# Sort the complexity scores in descending order
sorted_by_complexity = dict(sorted(complexity_scores.items(), key=lambda item: item[1], reverse=True))

# Calculate the percentiles based on the sorted complexity scores
# Higher complexity will have higher rank and thus higher percentile
complexity_ranks = rankdata(list(sorted_by_complexity.values()), method='average')
topic_percentiles = {topic: rank / len(complexity_scores) for topic, rank in zip(sorted_by_complexity.keys(), complexity_ranks)}

def get_turn_range(turns):
    """
    Converts a target turn number into a soft target range.

    Parameters:
    - turns (int): Target turn number.

    Returns:
    - A tuple representing the soft target range (min_turns, max_turns).
    """
    min_turns = max(1, turns - 1)
    max_turns = min(15, turns + 3)
    return min_turns, max_turns

def biased_normal_distribution(input_number):
    """
    Samples a number from a normal distribution that is biased towards the lower end,
    with a specified range and standard deviation.

    Parameters:
    - input_number (int or float): A number, typically representing a percentile, between 0 and 100 that
      influences the mean of the normal distribution.

    Returns:
    - int: A sample from the biased normal distribution, truncated to the nearest integer.
    """
    # Constants for the distribution
    min_val, max_val = 1, 15
    std_dev = 2.5  # Fixed standard deviation

    # Base mean for input_number 50 (biased towards lower end)
    base_mean = 1

    # Adjust the mean based on the input number (linear mapping)
    mean = np.interp(input_number, [0, 100], [min_val, max_val])
    
    # Adjust mean for the bias towards lower numbers
    adjusted_mean = (mean + 3*base_mean) / 5

    # Sample from a normal distribution
    while True:
        sample = np.random.normal(adjusted_mean, std_dev)
        if min_val <= sample <= max_val:
            break

    return int(sample)

for p, t in zip(problems, problem_titles):

    metadata_topic_hierarchy = p["metadata"].get("topic") 
    normalized_topic = metadata_topic_hierarchy.replace(" > ", ".")
    print(normalized_topic)

    # Retrieve the percentile using the normalized topic string
    percentile = topic_percentiles.get(normalized_topic, 0)
    print(f"Topic: {normalized_topic}, Percentile: {percentile}")

    # Calculate turns using the biased_normal_distribution function
    turns = biased_normal_distribution(percentile * 100)  # Percentile scaled to 0-100 
    print('turns'+str(turns))

    min_turns, max_turns = get_turn_range(turns)
    print(f'Target Number of Turns: {min_turns}-{max_turns}')
    
    # Create a new notebook
    notebook = new_notebook()
    # Add metadata
    metadata = f"""# Metadata

**Python Topics** - {p["metadata"]["topic"]}

**Type** - {p["metadata"]["type"]}

**Target Number of Turns (User + Assistant)** - {min_turns}-{max_turns}
"""
    metadata_cell = new_markdown_cell(metadata)
    notebook.cells.append(metadata_cell)

    # Add conversation header
    conversation_header = "# Conversation"
    conversation_header_cell = new_markdown_cell(conversation_header)
    notebook.cells.append(conversation_header_cell)

    # Append conversation messages
    user_message = f"""**User**

{p["messages"][0]["content"]}
"""
    conversation_message_cell = new_markdown_cell(user_message)
    notebook.cells.append(conversation_message_cell)

    # Save the notebook
    notebook_path = f'notebooks/v3/old_pipe/{t}.ipynb'
     
    try:
        os.makedirs(os.path.dirname(notebook_path))
    except FileExistsError:
        pass
     
    with open(notebook_path, 'w') as f:
        nbformat.write(notebook, f)

algorithms.by_topic.bit_manipulation
Topic: algorithms.by_topic.bit_manipulation, Percentile: 0.5378787878787878
turns9
Target Number of Turns: 8-12
algorithms.by_topic.bit_manipulation
Topic: algorithms.by_topic.bit_manipulation, Percentile: 0.5378787878787878
turns1
Target Number of Turns: 1-4
algorithms.by_topic.bit_manipulation
Topic: algorithms.by_topic.bit_manipulation, Percentile: 0.5378787878787878
turns2
Target Number of Turns: 1-5
algorithms.by_topic.math
Topic: algorithms.by_topic.math, Percentile: 0.5378787878787878
turns2
Target Number of Turns: 1-5
algorithms.by_topic.math
Topic: algorithms.by_topic.math, Percentile: 0.5378787878787878
turns2
Target Number of Turns: 1-5
algorithms.by_topic.math
Topic: algorithms.by_topic.math, Percentile: 0.5378787878787878
turns2
Target Number of Turns: 1-5
algorithms.by_data_structure.stacks
Topic: algorithms.by_data_structure.stacks, Percentile: 0.14015151515151514
turns2
Target Number of Turns: 1-5
algorithms.by_data_structure.stacks


In [4]:
import csv

def parse_and_flatten_csv(input_csv_filepath, output_csv_filepath):
    with open(input_csv_filepath, mode='r', newline='', encoding='utf-8') as infile, \
         open(output_csv_filepath, mode='w', newline='', encoding='utf-8') as outfile:
        
        reader = csv.reader(infile)
        writer = csv.writer(outfile)
        
        for row in reader:
            highlevel_topic = row[0].strip()
            subtopics = row[1].split('\n')
            for subtopic in subtopics:
                subtopic_cleaned = subtopic.strip()
                if subtopic_cleaned:
                    writer.writerow([highlevel_topic, subtopic_cleaned])

input_csv_filepath = './Copy of Copy of c.ai Turing Feedback - Areas to focus on.csv'  
output_csv_filepath = 'flattened.csv'

parse_and_flatten_csv(input_csv_filepath, output_csv_filepath)

In [50]:
import json
import csv


def process_sublevels(highlevel_topic, sublevels_dict, writer):
    for sublevel, nested in sublevels_dict.items():
        combined_sublevel = sublevel 
        if nested:
            for nested_topic in nested.keys():
                writer.writerow([highlevel_topic, f'{combined_sublevel}.{nested_topic}'])
        else:  
            writer.writerow([highlevel_topic, combined_sublevel])


with open('./topic_hierarchy.json', 'r') as json_file:
    data = json.load(json_file)


with open('topics_1.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)

    writer.writerow(['highlevel_topic', 'sublevel_topic'])


    for highlevel_topic, sublevels in data.items():
        if sublevels: 
            process_sublevels(highlevel_topic, sublevels, writer)
        else:  
            writer.writerow([highlevel_topic, ''])

In [77]:
df = pd.read_csv('./500.csv')
df

,https://colab.research.google.com/drive/1rFx3Q2Svj5NhLo_9yxen-sC0DRD-e0gj,Advanced_Debugging_and_Troubleshooting_Debug_error_trace__0_V3_B.ipynb
0,https://colab.research.google.com/drive/102x1y...,Advanced_Debugging_and_Troubleshooting_Debug_e...
1,https://colab.research.google.com/drive/1ziWkL...,Advanced_Debugging_and_Troubleshooting_Debug_e...
2,https://colab.research.google.com/drive/1FOAZw...,Advanced_Debugging_and_Troubleshooting_Debug_e...
3,https://colab.research.google.com/drive/1JHlDy...,Advanced_Debugging_and_Troubleshooting_Debug_e...
4,https://colab.research.google.com/drive/1YZP8v...,Advanced_Debugging_and_Troubleshooting_Debug_e...
...,...,...
495,https://colab.research.google.com/drive/1OFaBb...,Intermediate_Learning_Learn_to_use_some_tool__...
496,https://colab.research.google.com/drive/1qheya...,Intermediate_Write_unit_test_Explain_code_with...
497,https://colab.research.google.com/drive/1XaQdN...,Intermediate_Write_unit_test_Explain_code_with...
498,https://colab.research.google.com/drive/1l_uDM...,Intermediate_Write_unit_test_Explain_code_with...


In [24]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import csv

# Path to your service account key file
SERVICE_ACCOUNT_FILE = 'creds/google__sa.json'

# The scopes required for the Sheets API
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# The ID of your spreadsheet
SPREADSHEET_ID = '1qBU7Kvuuij2fxbqPxebReKMxWgIBmOIE5Gi4ZuX0j_4'

# Authenticate and build the service
creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('sheets', 'v4', credentials=creds)

# Specify the range and values to update
range_ = 'Conversations_Batch_6!A1:E1000' 

def read_csv(filename):
    with open(filename, newline='') as csv_file:
        reader = csv.reader(csv_file)
        return list(reader)

# Call the Sheets API
service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()

# Load data from the CSV file
csv_values = read_csv('./notebooks/v3/new_pipe/batch_2/batch_2.csv')

# Use the Sheets API to write data
request = sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=range_,
                               valueInputOption='RAW', body={'values': csv_values})
response = request.execute()

print(f"{len(csv_values) - 1} rows have been updated.")

500 rows have been updated.


In [15]:
with open('./topic_dist.json') as f:
    data = json.load(f)

# Flatten the data structure, creating a list of (category, complexity) tuples
def flatten_data(data, parent_key='', sep='_'):
    items = []
    for k, v in data.items():
        new_key = f'{parent_key}{sep}{k}' if parent_key else k
        if isinstance(v, dict):
            if 'complexity' in v:
                items.append((new_key, v['complexity']))
            else:
                items.extend(flatten_data(v, new_key, sep=sep))
    return items

# Flatten the provided JSON data
flattened_data = flatten_data(data)

# Now we sort the flattened data by complexity in descending order
sorted_data = sorted(flattened_data, key=lambda x: x[1], reverse=True)

for item in sorted_data:
    print(f"{item[0]}: Complexity {item[1]}")

algorithms_by_data_structure_advanced_data_structures: Complexity 8
algorithms_by_topic_dynamic_programming: Complexity 8
python_language_and_scripting_python_c_extensions: Complexity 8
deep_learning_backpropagation_understanding: Complexity 8
algorithms_by_data_structure_graphs: Complexity 7
algorithms_by_topic_famous_algorithms: Complexity 7
algorithms_by_topic_game_theory: Complexity 7
algorithms_by_topic_divide_and_conquer: Complexity 7
algorithms_by_topic_backtracking: Complexity 7
algorithms_by_topic_combinatorics: Complexity 7
python_language_and_scripting_metaclasses_and_class_factories: Complexity 7
python_language_and_scripting_memory_management_and_python_internals: Complexity 7
python_language_and_scripting_advanced_networking: Complexity 7
python_language_and_scripting_cython_and_PyPy: Complexity 7
python_language_and_scripting_parallel_programming: Complexity 7
unit_testing_methodology_test_ai_and_ml_models: Complexity 7
web_development_web_security: Complexity 7
ml_princ

In [19]:
import json

with open('notebooks/test/problems.json', 'w') as f:
    json.dump(problems, f, indent=4)


In [20]:
with open('notebooks/test/problems.json') as f:
    problems = json.load(f)

In [3]:
import os
import nbformat

def modify_markdown_metadata(notebook_path):
    # Load the notebook
    with open(notebook_path, 'r', encoding='utf-8') as f:
        notebook = nbformat.read(f, as_version=4)

    # Iterate through each cell and find markdown cells
    for cell in notebook.cells:
        if cell.cell_type == 'markdown':
            # We have a markdown cell; let's modify it as needed
            cell_text = cell.source

            # Check for the specific metadata line and modify it
            if "**Occupation Topics** -" in cell_text:
                lines = cell_text.split('\n')
                for i, line in enumerate(lines):
                    if line.startswith("**Occupation Topics** -"):
                        # Split the content at '>'
                        parts = line.split('>')
                        # Keep the part before '>'
                        lines[i] = parts[0].strip() if len(parts) > 1 else line
                # Update the cell source
                cell.source = '\n'.join(lines)
            
    # Save the notebook back to the file
    with open(notebook_path, 'w', encoding='utf-8') as f:
        nbformat.write(notebook, f)

def modify_all_notebooks_in_folder(folder_path):
    # Iterate over files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.ipynb'):
            notebook_path = os.path.join(folder_path, filename)
            modify_markdown_metadata(notebook_path)
            print(f'Modified notebook: {filename}')

# Set the folder to the path where your notebooks are stored
folder_path = './notebooks/v3/new_pipe/batch_1/'  # Update this to your folder path
modify_all_notebooks_in_folder(folder_path)

Modified notebook: Advanced Basic scripting - Write simple code in Javascript__0_V3_B.ipynb
Modified notebook: Advanced Debugging and Troubleshooting - Debug error trace__0_V3_B.ipynb
Modified notebook: Advanced Debugging and Troubleshooting - Debug error trace__10_V3_B.ipynb
Modified notebook: Advanced Debugging and Troubleshooting - Debug error trace__11_V3_B.ipynb
Modified notebook: Advanced Debugging and Troubleshooting - Debug error trace__12_V3_B.ipynb
Modified notebook: Advanced Debugging and Troubleshooting - Debug error trace__13_V3_B.ipynb
Modified notebook: Advanced Debugging and Troubleshooting - Debug error trace__14_V3_B.ipynb
Modified notebook: Advanced Debugging and Troubleshooting - Debug error trace__1_V3_B.ipynb
Modified notebook: Advanced Debugging and Troubleshooting - Debug error trace__2_V3_B.ipynb
Modified notebook: Advanced Debugging and Troubleshooting - Debug error trace__3_V3_B.ipynb
Modified notebook: Advanced Debugging and Troubleshooting - Debug error tra

In [104]:
from tqdm import tqdm
from src.gdrive_api.folder_upload import upload_folder
from src.gdrive_api.auth import build_service

service = build_service('creds/google__sa.json')
# destination_folder_url = "https://drive.google.com/drive/u/0/folders/1Z1bdYMe2Qmo_vs-OaKDaYIiV3rIqLJH9"
# destination_folder_url = "https://drive.google.com/drive/u/2/folders/1sfPFHkXYpKyY41V0pfz3Qw3k4VLy5Hvb"
# destination_folder_url = 'https://drive.google.com/drive/folders/1T14cHENpx7j2aS4Yb_3WhnXhMql0zq2y'
# destination_folder_url = 'https://drive.google.com/drive/u/2/folders/1jV7WA5zB172DJUp7Z2XzHr62E6U6_NtY'
destination_folder_url =  'https://drive.google.com/drive/u/2/folders/1FuHZZ18qn6k8iXTi5YbstGRr56w0kDEz'

uploaded_files = upload_folder(service, './notebooks/v3/new_pipe/batch_1/', destination_folder_url, force_replace = True, is_url=True)

file_path_to_url = {}
with tqdm(total=len(uploaded_files)) as pbar:
    for file_path, file_url in uploaded_files.items():
        if file_url is not None:
            drive_id = file_url.split("id=")[-1].split("&")[0].strip()
            colab_url = f"https://colab.research.google.com/drive/{drive_id}"
            file_path_to_url[file_path] = colab_url
        else:
            print(f"Skipped uploading {file_path}")
        pbar.update(1)

for file_path in file_path_to_url.keys():
    if file_path == "problems.json":
        continue
    problem = file_path_to_problem[file_path]
    problem["metadata"]["colab_url"] = file_path_to_url[file_path]
    problem["metadata"]["file_path"] = file_path
    problem["metadata"]["batch_idx"] = "batch_1"

------------------------------------------------------------
Processing directory .: 1 of 0 in total.
Uploading file 1 of 500 in '.', 1 of 500 in total.
Uploading new file 'advanced_data_structures__modification__0_V3_A.ipynb'.
File 'advanced_data_structures__modification__0_V3_A.ipynb' has been uploaded.
Uploaded 'advanced_data_structures__modification__0_V3_A.ipynb' to folder ID '1FuHZZ18qn6k8iXTi5YbstGRr56w0kDEz'.
advanced_data_structures__modification__0_V3_A.ipynb
Uploading file 2 of 500 in '.', 2 of 500 in total.
Uploading new file 'advanced_data_structures__modification__1_V3_A.ipynb'.
File 'advanced_data_structures__modification__1_V3_A.ipynb' has been uploaded.
Uploaded 'advanced_data_structures__modification__1_V3_A.ipynb' to folder ID '1FuHZZ18qn6k8iXTi5YbstGRr56w0kDEz'.
advanced_data_structures__modification__1_V3_A.ipynb
Uploading file 3 of 500 in '.', 3 of 500 in total.
Uploading new file 'advanced_data_structures__modification__2_V3_A.ipynb'.
File 'advanced_data_structur

100%|██████████| 500/500 [00:00<00:00, 138361.94it/s]


In [106]:
problems = list(file_path_to_problem.values())

with open('notebooks/v3/old_pipe/problems.json', 'w') as f:
    json.dump(problems, f, indent=4)

In [108]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Path to your service account key file
SERVICE_ACCOUNT_FILE = 'creds/google__sa.json'

# The scopes required for the Sheets API
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# The ID of your spreadsheet
SPREADSHEET_ID = '1qBU7Kvuuij2fxbqPxebReKMxWgIBmOIE5Gi4ZuX0j_4'


# Authenticate and build the service
creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('sheets', 'v4', credentials=creds)

# Specify the range and values to update
range_ = 'Conversations_Batch_5!A380:E1000'  # For example, this updates cells from A1 to D5 in Sheet1
values = []

for problem in problems:
    values.append([
        problem["metadata"]["colab_url"],
        problem["metadata"]["topic"],
    ])


body = {
    'values': values
}

# Call the Sheets API to update the range
request = service.spreadsheets().values().update(spreadsheetId=SPREADSHEET_ID, range=range_, valueInputOption='RAW', body=body)
response = request.execute()

In [107]:
problems

[{'metadata': {'topic': 'algorithms > by_topic > bit_manipulation',
   'type': 'query',
   'difficulty': 'Easy',
   'target_length': 1,
   'colab_url': 'https://colab.research.google.com/drive/11gR4WH4YasYWimtQzUQ2h2ayG2JvxZK2',
   'file_path': 'bit_manipulation__query__0_V3_A.ipynb',
   'batch_idx': 'old_pipe'},
  'messages': [{'role': 'user',
    'content': 'Can you show me how to set the nth bit of a number to 1 in Python?'}]},
 {'metadata': {'topic': 'algorithms > by_topic > bit_manipulation',
   'type': 'query',
   'difficulty': 'Easy',
   'target_length': 1,
   'colab_url': 'https://colab.research.google.com/drive/1S-tOO61E8nFwh3SoMgHjTuwIAy8DQNEG',
   'file_path': 'bit_manipulation__query__1_V3_A.ipynb',
   'batch_idx': 'old_pipe'},
  'messages': [{'role': 'user',
    'content': 'How do I use bitwise operators to determine if all the bits of a number are set to 1?'}]},
 {'metadata': {'topic': 'algorithms > by_topic > bit_manipulation',
   'type': 'query',
   'difficulty': 'Easy'

In [ ]:
# python related questions, add - student related types

#### create occupation with experience levels

In [5]:
import json

with open('./occupations.json') as json_file:
    topic_hierarchy = json.load(json_file)

def combine_occupations_with_experience(occupations_dict, experience_levels):
    occupation_pairs = []
    for occupation_category in occupations_dict:
        if occupation_category == "UserExperience":  # Skip experience levels key
            continue

        for occupation_type in ["MostPopular", "Trending"]:
            for occupation in occupations_dict[occupation_category].get(occupation_type, []):
                for level in experience_levels:
                    occupation_pairs.append(f"{occupation} > {level}")
    return occupation_pairs

# Extract the experience levels from the hierarchy
experience_levels = topic_hierarchy["UserExperience"]["Levels"]

# Combine occupations with experience levels
occupation_with_experience_pairs = combine_occupations_with_experience(topic_hierarchy, experience_levels)

print(f"Total number of occupation with experience levels: {len(occupation_with_experience_pairs)}")
print(occupation_with_experience_pairs)

Total number of occupation with experience levels: 200
['Python Developer > First Timer', 'Python Developer > Beginner', 'Python Developer > Intermediate', 'Python Developer > Advanced', 'Python Developer > Expert', 'Web Developer > First Timer', 'Web Developer > Beginner', 'Web Developer > Intermediate', 'Web Developer > Advanced', 'Web Developer > Expert', 'Data Scientist > First Timer', 'Data Scientist > Beginner', 'Data Scientist > Intermediate', 'Data Scientist > Advanced', 'Data Scientist > Expert', 'Mobile App Developer > First Timer', 'Mobile App Developer > Beginner', 'Mobile App Developer > Intermediate', 'Mobile App Developer > Advanced', 'Mobile App Developer > Expert', 'Python DevOps Engineer > First Timer', 'Python DevOps Engineer > Beginner', 'Python DevOps Engineer > Intermediate', 'Python DevOps Engineer > Advanced', 'Python DevOps Engineer > Expert', 'Front-End Developer > First Timer', 'Front-End Developer > Beginner', 'Front-End Developer > Intermediate', 'Front-End

In [6]:
occupation_skills = occupation_with_experience_pairs
occupation_skills

['Python Developer > First Timer',
 'Python Developer > Beginner',
 'Python Developer > Intermediate',
 'Python Developer > Advanced',
 'Python Developer > Expert',
 'Web Developer > First Timer',
 'Web Developer > Beginner',
 'Web Developer > Intermediate',
 'Web Developer > Advanced',
 'Web Developer > Expert',
 'Data Scientist > First Timer',
 'Data Scientist > Beginner',
 'Data Scientist > Intermediate',
 'Data Scientist > Advanced',
 'Data Scientist > Expert',
 'Mobile App Developer > First Timer',
 'Mobile App Developer > Beginner',
 'Mobile App Developer > Intermediate',
 'Mobile App Developer > Advanced',
 'Mobile App Developer > Expert',
 'Python DevOps Engineer > First Timer',
 'Python DevOps Engineer > Beginner',
 'Python DevOps Engineer > Intermediate',
 'Python DevOps Engineer > Advanced',
 'Python DevOps Engineer > Expert',
 'Front-End Developer > First Timer',
 'Front-End Developer > Beginner',
 'Front-End Developer > Intermediate',
 'Front-End Developer > Advanced',
 'F

In [72]:
print(type(occupation_with_experience_pairs))

<class 'list'>


In [7]:
df = pd.read_csv('./compatible_area.csv')

filtered_df = df[df['Status'] == 'To Cover']

print(len(filtered_df))

17


In [8]:
filtered_df['combined'] = filtered_df['Highlevel_topic'] + " - " + filtered_df['Sublevel_topic']

C:\Users\hp\AppData\Local\Temp\ipykernel_30224\2856107563.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['combined'] = filtered_df['Highlevel_topic'] + " - " + filtered_df['Sublevel_topic']


In [9]:
areas_of_focus = filtered_df['combined'].to_list()

In [10]:
areas_of_focus

['Basic scripting - Write simple code in Javascript',
 'General coding help - Rewrite code using better formatting / best practises',
 'General coding help - Refactor code',
 'General coding help - Explain complex code snippets',
 'General coding help - Add comments to code',
 'General coding help - Do a code review',
 'Write unit test - Explain code with comments',
 'Interview preparation - Explain the problem step by step',
 'Interview preparation - Explain areas where people usually make mistakes',
 'Interview preparation - Explain alternative solutions and trade offs',
 'Interview preparation - Debug and fix code',
 'Interview preparation - System Design rounds',
 'Interview preparation - Proactively ask follow up questions',
 'Interview preparation - User asks AI help to take an Interview. AI takes an interview',
 'Debugging and Troubleshooting - Debug error trace',
 'Learning - Learn to program',
 'Learning - Learn to use some tool / infra']

In [ ]:
# 3. Suggestions should provide a clear direction for further development or learning.

#### matching Occupation and skill pairs with Compatible areas of focus

In [103]:
# def compatible_area_check(occupation_skill, area_of_focus, n=1, existing_questions=None):
#     SYSTEM_PROMPT = f"""IDENTITY:
# You are an AI capable of identifying compatible areas of focus based on individuals' occupation and skills from the given list of compatible area only and if not compatible with occupation and skill from the list of area of focus please skip and only include the ones compatible..

# CONTEXT:
# We are trying to assist users in aligning their professional expertise and their skills with the most compatible areas of focus. This will help in furthering their career development and staying relevant in their fields.
# We already have a batch of existing occupation-skill-area of focus combinations and we want to avoid duplication when providing new suggestions.

# CONSTRAINTS:
# 1. The suggestions must be from the input list of compatible area relevant to the provided occupation and skill level if not compatible with occupation and skill from the list of area of focus please skip and only include the ones compatible..
# 2. It should be practical, meaning it can realistically be pursued given the user's background.


# INSTRUCTION:
# You will be given pairs of occupations and skills and and also list of selected compatible areas and will be asked to find a number of compatible areas of focus. You should also be provided with existing pairs and asked to ignore suggesting those if they already exist.
# Act accordingly.

# Find compatible areas of focus for the given occupation and skill pairs:
# - Occupation and skill pairs: 
# The suggestions should only be from the provided areas of focus if not compatible with occupation and skill from the list of area of focus please skip and only include the ones compatible.

# RESPONSE FORMAT:
# A JSON-valid object that includes {{"questions": ['Occupation and skill','area_of_focus1', 'Occupation and skill','area_of_focus2', ...] }}.
# """
#     try:
#         response = client.chat.completions.create(
#             model="gpt-4-1106-preview",
#             messages=[
#                 {"role": "system", "content": SYSTEM_PROMPT},
#                 {"role": "user", "content": f"Occupation and skill: {occupation_skill} \nCompatible areas of focus: {area_of_focus}"},
#             ],
#             temperature=0.0,
#             max_tokens=4096,
#             seed = 42,
#             response_format={ 
#                 "type": "json_object" 
#             },
#         )
#         questions = json.loads(response.choices[0].message.content)
#         return questions
#     except Exception as e:
#         print(f"An error occurred: {e}")
#         return None

In [2]:
def compatible_area_check(occupation_skill, areas_of_focus, existing_questions=None):
    SYSTEM_PROMPT = f"""IDENTITY:
You are an AI capable of identifying compatible areas of focus based on individuals' occupation and skills from the given list of compatible area only and if not compatible with occupation and skill from the list of area of focus please skip and only include the ones compatible..

CONTEXT:
We are trying to assist users in aligning their professional expertise and their skills with the most compatible areas of focus. This will help in furthering their career development and staying relevant in their fields.
We already have a batch of existing occupation-skill-area of focus combinations and we want to avoid duplication when providing new suggestions.

CONSTRAINTS:
1. The suggestions must be from the input list of compatible area relevant to the provided occupation and skill level if not compatible with occupation and skill from the list of area of focus please skip and only include the ones compatible.
2. It should be practical, meaning it can realistically be pursued given the user's background.

INSTRUCTION:
You will be provided with an occupation and skill level and a list of potential areas of focus. You must evaluate which of the areas focus are compatible with the given occupation and skill level, and suggest them. Ignore suggesting areas of focus that are already listed in the existing questions if provided.
Act accordingly.

Occupation and skill pair: {occupation_skill} 
Potential areas of focus: {', '.join(areas_of_focus)}
Existing combinations: {existing_questions if existing_questions else 'None provided'}

RESPONSE FORMAT:
A JSON object that includes compatible areas of focus for the given occupation-skill pair.

Example Call:
Occupation and skill pair: 'Python Developer > Intermediate'
Potential areas of focus: Basic scripting, Write unit tests, Learn to program, General coding help - Explain complex code snippets
Existing combinations: None provided

Response:
{{"compatible_areas": ["Learn to program", "General coding help - Explain complex code snippets"]}}
"""
    try:
        response = client.chat.completions.create(
            model="gpt-4-1106-preview",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": f"Find the compatible areas of focus for the provided occupation and skill pair."}
            ],
            temperature=0.0,
            max_tokens=4096,
            stop=None,
            seed=42
        )

        questions = json.loads(response.choices[0].message.content) if response.choices[0].message.content else {}
        return questions

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [ ]:
with tqdm(total=len(occupation_skills)) as pbar:
    for occupation_skill in occupation_skills:
        matched_areas = compatible_area_check(occupation_skill, areas_of_focus, existing_questions)

        if matched_areas and "compatible_areas" in matched_areas:
            for area_of_focus in matched_areas["compatible_areas"]:
                print(occupation_skill, area_of_focus)
                problems.append({
                    "metadata": {
                        "occupation_skill": occupation_skill,
                        "compatible_area_of_focus": area_of_focus,
                        "type": "query",
                        "difficulty": "Easy",
                        "target_length": 1
                    },
                    "messages": [
                        {"role": "user", "content": f"{occupation_skill} matched with {area_of_focus}"},
                    ]
                })
        pbar.update(1)

In [84]:
with open('./occMatch.txt', 'r') as file:
    lines = file.readlines()

In [85]:
lines[:10]

['Python Developer > First Timer Basic scripting - Write simple code in Javascript\n',
 'Python Developer > First Timer General coding help - Rewrite code using better formatting / best practises\n',
 'Python Developer > First Timer General coding help - Refactor code\n',
 'Python Developer > First Timer General coding help - Explain complex code snippets\n',
 'Python Developer > First Timer General coding help - Add comments to code\n',
 'Python Developer > First Timer General coding help - Do a code review\n',
 'Python Developer > First Timer Write unit test - Explain code with comments\n',
 'Python Developer > First Timer Interview preparation - Explain the problem step by step\n',
 'Python Developer > First Timer Interview preparation - Explain areas where people usually make mistakes\n',
 'Python Developer > First Timer Interview preparation - Explain alternative solutions and trade offs\n']

In [86]:
cleaned_list = [item.strip() for item in lines]

print(len(cleaned_list))

2188


In [ ]:
# total_iterations = len(occupation_skills) * len(areas_of_focus), [postprocessing phase]stratify across area of focus

In [ ]:
# problems = []


# with tqdm(total=len(occupation_skills)) as pbar:
#     for occupation_skill in occupation_skills:
#         for area_of_focus in areas_of_focus:

#             matched_area = compatible_area_check(occupation_skill, area_of_focus, n=1)

#             for question in matched_area["questions"]:
#                 print(question)
#                 problems.append({
#                     "metadata": {
#                         "occupation_skill": occupation_skill,
#                         "compatible_area_of_focus": area_of_focus,
#                         "type": "query",
#                         "difficulty": "Easy",
#                         "target_length": 1
#                     },
#                     "messages": [
#                         {"role": "user", "content": question},
#                     ]
#                 })
#                 pbar.update(1)

In [87]:
def flatten_json(json_data):
    flattened_data = []
    
    for entry in json_data:
        # Extract the metadata and messages
        metadata = entry['metadata']
        messages = entry['messages']
        
        # Combine metadata with each message
        for message in messages:
            flattened = {**metadata, **message}
            flattened_data.append(flattened)
    
    return flattened_data

In [ ]:
flattened_json_data = flatten_json(problems)

for item in flattened_json_data:
    print(item)

#### generating user themes based on their occupation_experience level and compatible area of focus

In [89]:
def generate_usecase_technical_topic(occupation, n=1, existing_questions=None):
    SYSTEM_PROMPT = f"""IDENTITY:
You are an AI capable of generating user themes based on their occupation_experience level and compatible area of focus.

CONTEXT:
We are trying to generate human-like queries that a user would send to an ai assistant through a chat interface. 
The user's query tone & structure should be diversified as much as possible making sure to include some realistic examples.
We already have batch of previously generated questions and we want to avoid duplication of questions when generating new batch.

CONSTRAINTS:
1. Occupation and skill Related
2. medium (Solvable by a median developer in 60 minutes)
3. Questions should elicit a response that includes code.

INSTRUCTION:
You will be given an occupation_experience level and compatible area of focus and asked for number of Use Case coupled with Technical Topic and User Personality should not be repetitive it should be genuine for each user to generate.
You will also be given previously generated questions for respective topic and ask to ignore generating question if exist.
The aim is to maximize diversity.
Act accordingly.

Generate a theme object for a user with the following occupation_experience level and compatible area of focus:
- Occupation: 
- Experience level:
- compatible area of focus: 

The theme object should contain a use case, a technical topic, and the user's personality.

A technical topic is a narrowly defined subject area concentrating on a specific library, feature, coding or design technique mostly within the Python programming language, rarely other languages too.

User personality dimention ideas, Atleast use three of the scales given below (optimize diversity) should be a brief english description:
Analytical vs. Intuitive
Curious vs. Focused
Engaged vs. Detached
Confident vs. Hesitant
Open-Minded vs. Opinionated
Concise vs. Elaborate
Practical vs. Theoretical
Patient vs. Impatient
Collaborative vs. Independent
Formal vs. Informal

RESPONSE FORMAT:
A JSON-valid object that includes {{"questions":["use_case", "technical_topic", and "user_personality"]}}.

"""
    try:
        response = client.chat.completions.create(
            model="gpt-4-1106-preview",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": f"occupation_experience level and compatible area of focus: {occupation} \nNumber of questions: {n} \n existing questions {existing_questions}"},
            ],
            temperature=0.0,
            max_tokens=4096,
            seed = 42,
            response_format={ 
                "type": "json_object" 
            },
        )
        questions = json.loads(response.choices[0].message.content)
        return questions
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [90]:
MAX_QUESTIONS = 500

In [ ]:
# generator = QuestionGenerator()
# questions = generator.generate_human_like_questions("Python", 5)

In [91]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from src.Q_generator.question_generator
from tqdm import tqdm

generated_questions_count = 0  # Initialize your question count

problems = []

# Define the function to be executed in the thread
def generate_questions_for_occupation(Occupation):
    global generated_questions_count
    occupation_problems = []

    if generated_questions_count < MAX_QUESTIONS:
        questions = generate_usecase_technical_topic(Occupation, 3)
        for question in questions["questions"]:
            if generated_questions_count >= MAX_QUESTIONS:
                return occupation_problems  # Stop if the limit is reached
            occupation_problems.append({
                "metadata": {
                    "Occupation": Occupation,
                },
                "messages": [
                    {"role": "user", "content": question},
                ]
            })
            generated_questions_count += 1  # Update the question count

    return occupation_problems

# Set the maximum number of threads to use
MAX_THREADS = 30

# Use ThreadPoolExecutor to run the tasks asynchronously
with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    # Create a list to store future objects
    future_to_occupation = [executor.submit(generate_questions_for_occupation, occupation) for occupation in cleaned_list]

    with tqdm(total=MAX_QUESTIONS) as pbar:
        for future in as_completed(future_to_occupation):
            new_problems = future.result()
            problems.extend(new_problems)
            # Update the progress bar by the number of new problems generated
            pbar.update(len(new_problems))

print(f"Total questions generated: {len(problems)}")


100%|██████████| 500/500 [01:40<00:00,  4.99it/s]

Total questions generated: 500


In [92]:
problems

[{'metadata': {'Occupation': 'Python Developer > First Timer Learning - Learn to use some tool / infra'},
  'messages': [{'role': 'user',
    'content': {'use_case': 'Setting up a Python development environment for the first time',
     'technical_topic': 'Virtual environments with virtualenv',
     'user_personality': 'Curious, patient, and independent'}}]},
 {'metadata': {'Occupation': 'Python Developer > First Timer Learning - Learn to use some tool / infra'},
  'messages': [{'role': 'user',
    'content': {'use_case': 'Writing a basic Python script to automate a simple task',
     'technical_topic': 'File I/O operations in Python',
     'user_personality': 'Engaged, confident, and practical'}}]},
 {'metadata': {'Occupation': 'Python Developer > First Timer Learning - Learn to use some tool / infra'},
  'messages': [{'role': 'user',
    'content': {'use_case': 'Understanding and handling errors in Python scripts',
     'technical_topic': 'Exception handling with try-except blocks',


In [10]:
data = problems
len(data)

6564

In [106]:
import random

def stratify_and_sample(data, stratify_key, sample_size):
    # Stratify data based on the stratify_key
    stratified = {}
    for item in data:
        key = item['metadata'][stratify_key]
        if key in stratified:
            stratified[key].append(item)
        else:
            stratified[key] = [item]

    # Calculate the number of samples per stratum based on their proportion
    total_count = len(data)
    samples_per_stratum = {key: int((len(value) / total_count) * sample_size) for key, value in stratified.items()}

    # Sample items from each stratum
    sampled = []
    for key, value in stratified.items():
        samples = random.sample(value, min(samples_per_stratum[key], len(value)))
        sampled.extend(samples)
        # Removing sampled items from the original data
        for sample in samples:
            data.remove(sample)

    # To ensure exact sample size, if needed due to rounding, add or remove some items
    while len(sampled) < sample_size:
        additional_samples = random.sample(data, 1)
        sampled.extend(additional_samples)
        for sample in additional_samples:
            data.remove(sample)

    while len(sampled) > sample_size:
        data.append(sampled.pop())

    return sampled

sampled_items = stratify_and_sample(data, 'Occupation', 500)

print(f"Selected {len(sampled_items)} items.")
print("Remaining data size:", len(data))

Selected 500 items.
Remaining data size: 64


In [98]:
import json
import os

sampled_json = json.dumps(problems, indent=4)

folder_path = './notebooks/v3/new_pipe/v3_1'
file_name = 'problems.json'
file_path = os.path.join(folder_path, file_name)

os.makedirs(folder_path, exist_ok=True)

with open(file_path, 'w') as file:
    file.write(sampled_json)

print(f"Selected {len(problems)} items and saved to {file_path}.")
print("Remaining data size:", len(problems))

Selected 500 items and saved to ./notebooks/v3/new_pipe/v3_1\problems.json.
Remaining data size: 500


In [367]:
len(data)

64

In [79]:
len(sampled_items)

500

In [112]:
# problems = []

# with tqdm(total=MAX_QUESTIONS) as pbar:
#     for Occupation in cleaned_list:
#         # Stop generating further questions if we've reached the max limit
#         if generated_questions_count >= MAX_QUESTIONS:
#             break

#         # Generate 1 question per occupation
#         questions = generate_usecase_technical_topic(Occupation, 1)

#         print(questions)

#         for question in questions["questions"]:
#             if generated_questions_count >= MAX_QUESTIONS:
#                 break

#             problems.append({
#                 "metadata": {
#                     "Occupation": Occupation,
#                     "type": "query",
#                     "difficulty": "Easy",
#                     "target_length": 1
#                 },
#                 "messages": [
#                     {"role": "user", "content": question},
#                 ]
#             })
#             generated_questions_count += 1
#             pbar.update(1)

# print(f"Total questions generated: {len(problems)}")

In [9]:
json_file_path = './notebooks/v3/new_pipe/batch_2/problems.json'

with open(json_file_path, 'r') as file:
    problems = json.load(file)
print(len(problems))

500


In [10]:
problem_titles = []
problem_topic_counts = {}
file_path_to_problem = {}
for problem in problems:
    occupation = problem["metadata"]["Occupation"]
    topic_type = f'{occupation.split(" > ")[-1]}'
    idx = problem_topic_counts.get(topic_type, 0)
    title = f'{topic_type}__{idx}'

    modified_title = f'{title}_V3_B'

    problem_titles.append(modified_title)
    file_path_to_problem[f"{modified_title}.ipynb"] = problem  # Use the modified title for the dictionary key
    problem_topic_counts[topic_type] = idx + 1

In [11]:
problem_titles

['First Timer Basic scripting - Write simple code in Javascript__0_V3_B',
 'Intermediate Interview preparation - Debug and fix code__0_V3_B',
 'Advanced General coding help - Explain complex code snippets__0_V3_B',
 'Advanced Interview preparation - Explain alternative solutions and trade offs__0_V3_B',
 'Expert General coding help - Explain complex code snippets__0_V3_B',
 'Intermediate Interview preparation - Debug and fix code__1_V3_B',
 'First Timer Learning - Learn to program__0_V3_B',
 'Beginner Interview preparation - System Design rounds__0_V3_B',
 'Expert General coding help - Add comments to code__0_V3_B',
 'Intermediate Interview preparation - Explain areas where people usually make mistakes__0_V3_B',
 'Expert General coding help - Rewrite code using better formatting - best practises__0_V3_B',
 'Beginner Interview preparation - Explain alternative solutions and trade offs__0_V3_B',
 'First Timer Basic scripting - Write simple code in Javascript__1_V3_B',
 'Advanced Intervie

In [12]:
len(file_path_to_problem)

500

In [18]:
import json
import nbformat
import os
import re
import numpy as np
from nbformat.v4 import new_notebook, new_markdown_cell
from scipy.stats import rankdata

def flatten_complexity_scores(data, key_prefix='', scores_dict=None):
    if scores_dict is None:
        scores_dict = {}
    for key, value in data.items():
        if isinstance(value, dict):
            if 'complexity' in value:
                scores_dict[f'{key_prefix}.{key}'.strip('.')] = value['complexity']
            else:
                new_prefix = f'{key_prefix}.{key}'.strip('.')
                flatten_complexity_scores(value, new_prefix, scores_dict)
        elif isinstance(value, int):
            scores_dict[key_prefix] = value
    return scores_dict

with open('./topic_dist_occupation.json') as f:
    complexity_data = json.load(f)

# Flatten the complexity data into a dictionary
complexity_scores = flatten_complexity_scores(complexity_data)

# Sort the complexity scores in descending order
sorted_by_complexity = dict(sorted(complexity_scores.items(), key=lambda item: item[1], reverse=True))

# Calculate the percentiles based on the sorted complexity scores
# Higher complexity will have higher rank and thus higher percentile
complexity_ranks = rankdata(list(sorted_by_complexity.values()), method='average')
topic_percentiles = {topic: rank / len(complexity_scores) for topic, rank in zip(sorted_by_complexity.keys(), complexity_ranks)}

# def sanitize_filename(filename):
#     filename = re.sub('[\/:*?"<>|]', '', filename)  # Remove problematic characters
#     filename = filename.replace(' - ', '_').replace(' ', '_')  # Replace spaces and ' - ' with underscores
#     return filename

def get_turn_range(turns):
    """
    Converts a target turn number into a soft target range.

    Parameters:
    - turns (int): Target turn number.

    Returns:
    - A tuple representing the soft target range (min_turns, max_turns).
    """
    min_turns = max(1, turns - 1)
    max_turns = min(15, turns + 3)
    return min_turns, max_turns

def biased_normal_distribution(input_number):
    """
    Samples a number from a normal distribution that is biased towards the lower end,
    with a specified range and standard deviation.

    Parameters:
    - input_number (int or float): A number, typically representing a percentile, between 0 and 100 that
      influences the mean of the normal distribution.

    Returns:
    - int: A sample from the biased normal distribution, truncated to the nearest integer.
    """
    # Constants for the distribution
    min_val, max_val = 1, 15
    std_dev = 2.5  # Fixed standard deviation

    # Base mean for input_number 50 (biased towards lower end)
    base_mean = 1

    # Adjust the mean based on the input number (linear mapping)
    mean = np.interp(input_number, [0, 100], [min_val, max_val])
    
    # Adjust mean for the bias towards lower numbers
    adjusted_mean = (mean + 3*base_mean) / 5

    # Sample from a normal distribution
    while True:
        sample = np.random.normal(adjusted_mean, std_dev)
        if min_val <= sample <= max_val:
            break

    return int(sample)

for p, t in zip(problems, problem_titles):

    metadata_occupation = p["metadata"].get("Occupation")
    normalized_topic = metadata_occupation.replace(" > ", ".")
    print(normalized_topic)

    # Retrieve the percentile using the normalized topic string
    percentile = topic_percentiles.get(normalized_topic, 0)
    print(f"Occupation: {normalized_topic}, Percentile: {percentile}")

    # Calculate turns using the biased_normal_distribution function
    turns = biased_normal_distribution(percentile * 100)  # Percentile scaled to 0-100 
    print('turns'+str(turns))

    min_turns, max_turns = get_turn_range(turns)
    print(f'Target Number of Turns: {min_turns}-{max_turns}')
    
    # Create a new notebook
    notebook = new_notebook()
    # Add metadata
    metadata = f"""# Metadata

**Occupation Topics** - {p["metadata"]["Occupation"]}

**Target Number of Turns (User + Assistant)** - {min_turns}-{max_turns}

**Use Case** - {p["messages"][0]["content"]["use_case"]}

**Technical Topic** - {p["messages"][0]["content"]["technical_topic"]}

**User Personality** - {p["messages"][0]["content"]["user_personality"]}
"""
    metadata_cell = new_markdown_cell(metadata)
    notebook.cells.append(metadata_cell)

    # Add conversation header
    conversation_header = "# Conversation"
    conversation_header_cell = new_markdown_cell(conversation_header)
    notebook.cells.append(conversation_header_cell)

    # Append conversation messages
    user_content = p["messages"][0]["content"]
    user_message = f"""**User**
"""
    conversation_message_cell = new_markdown_cell(user_message)
    notebook.cells.append(conversation_message_cell)

    # Save the notebook
    # sanitized_title = sanitize_filename(t)
    notebook_path = f'notebooks/v3/new_pipe/batch_2/{t}.ipynb'
        
    os.makedirs('notebooks/v3/new_pipe/batch_2', exist_ok=True)

    with open(notebook_path, 'w') as f:
        nbformat.write(notebook, f)

Product Manager.First Timer Basic scripting - Write simple code in Javascript
Occupation: Product Manager.First Timer Basic scripting - Write simple code in Javascript, Percentile: 0
turns1
Target Number of Turns: 1-4
Data Engineer.Intermediate Interview preparation - Debug and fix code
Occupation: Data Engineer.Intermediate Interview preparation - Debug and fix code, Percentile: 0
turns3
Target Number of Turns: 2-6
Data Visualization Developer.Advanced General coding help - Explain complex code snippets
Occupation: Data Visualization Developer.Advanced General coding help - Explain complex code snippets, Percentile: 0
turns4
Target Number of Turns: 3-7
Data Scientist.Advanced Interview preparation - Explain alternative solutions and trade offs
Occupation: Data Scientist.Advanced Interview preparation - Explain alternative solutions and trade offs, Percentile: 0
turns3
Target Number of Turns: 2-6
Robotics Engineer with Python Expertise.Expert General coding help - Explain complex code 

In [108]:
import json

with open('notebooks/v3/new_pipe/v3_1/problems.json', 'w') as f:
    json.dump(problems, f, indent=4)


In [109]:

with open('notebooks/v3/new_pipe/v3_1/problems.json') as f:
    sampled_items = json.load(f)

len(sampled_items)

500

In [110]:
sampled_items

[{'metadata': {'Occupation': 'Python Developer > First Timer Learning - Learn to use some tool - infra'},
  'messages': [{'role': 'user',
    'content': {'use_case': 'Setting up a Python development environment for the first time',
     'technical_topic': 'Virtual environments with virtualenv',
     'user_personality': 'Curious, patient, and independent'}}]},
 {'metadata': {'Occupation': 'Python Developer > First Timer Learning - Learn to use some tool - infra'},
  'messages': [{'role': 'user',
    'content': {'use_case': 'Writing a basic Python script to automate a simple task',
     'technical_topic': 'File I-O operations in Python',
     'user_personality': 'Engaged, confident, and practical'}}]},
 {'metadata': {'Occupation': 'Python Developer > First Timer Learning - Learn to use some tool - infra'},
  'messages': [{'role': 'user',
    'content': {'use_case': 'Understanding and handling errors in Python scripts',
     'technical_topic': 'Exception handling with try-except blocks',


In [136]:
# import os
# import threading
# from queue import Queue
# from tqdm import tqdm
# from src.gdrive_api.folder_upload import upload_file  # Assumption: this function is available for individual files
# from src.gdrive_api.auth import build_service

# # Service construction
# service = build_service('./creds/google__sa.json')
# destination_folder_url = 'https://drive.google.com/drive/u/2/folders/19dZkD6C9q52hFoSqn3e7clPDpN92L8uJ'

# # To prepare for threading, we use a Queue for thread-safe operations
# upload_jobs = Queue()

# # Directory listing
# folder_path = 'notebooks/v3/new_pipe/batch_1'
# files_to_upload = os.listdir(folder_path)
# pbar = tqdm(total=len(files_to_upload))
# file_path_to_url = {}
# file_path_to_problem = {}  # Define this as per your specific requirements

# # Function to run within each thread
# def worker():
#     while not upload_jobs.empty():
#         file_path = upload_jobs.get()
#         try:
#             file_url = upload_file(service, file_path, destination_folder_url, force_replace=True)
#             if file_url is not None:
#                 # Extract the file ID and create the Colab URL
#                 drive_id = file_url.split("id=")[-1].split("&")[0].strip()
#                 colab_url = f"https://colab.research.google.com/drive/{drive_id}"
#                 file_path_to_url[file_path] = colab_url
#             else:
#                 print(f"Skipped uploading {file_path}")
#         finally:
#             # Mark the job as done and update the progress bar
#             upload_jobs.task_done()
#             pbar.update(1)

# # Initialize the Queue with file paths
# for file_path in files_to_upload:
#     full_file_path = os.path.join(folder_path, file_path)
#     upload_jobs.put(full_file_path)

# # Number of threads
# num_threads = 20 

# # Start the threads
# threads = []
# for i in range(num_threads):
#     thread = threading.Thread(target=worker)
#     thread.start()
#     threads.append(thread)

# # Wait for all jobs to finish
# upload_jobs.join()

# # You might not need to wait for all threads here since join() above ensures all work is done, 
# # but it's good practice in case there are other non-queue-related tasks in the threads
# for thread in threads:
#     thread.join()

# pbar.close()

# # Post-processing: this will be executed after all threads have completed.
# for file_path in file_path_to_url.keys():
#     if file_path == "problems.json":
#         continue
#     if file_path in file_path_to_problem:
#         problem = file_path_to_problem[file_path]
#         problem["metadata"]["colab_url"] = file_path_to_url[file_path]
#         problem["metadata"]["file_path"] = file_path
#         problem["metadata"]["batch_idx"] = "new_pipe"
#     else:
#         print(f"Warning: No entry for '{file_path}' found in file_path_to_problem.")

In [71]:
import re

def extract_info(data):
    occupation_search = re.search(r'\*\*Occupation Topics\*\* - ([\w\s-]+(?:\(.*?\))?[ \w-]*) > ([\w\s-]+?) - (.+)', data)
    turns_search = re.search(r'\*\*Target Number of Turns \(User \+ Assistant\)\*\* - ([\d-]+)', data)
    use_case_search = re.search(r'\*\*Use Case\*\* - (.+)', data)
    technical_topic_search = re.search(r'\*\*Technical Topic\*\* - (.+)', data)
    personality_search = re.search(r'\*\*User Personality\*\* - (.+)', data)

    experience_levels = ["First Timer", "Beginner", "Intermediate", "Advanced", "Expert"]

    if occupation_search:
        occupation = occupation_search.group(1).strip()
        raw_experience_and_topic = occupation_search.group(2).strip()
        learning_topic = occupation_search.group(3).strip()

        # Match any of the predefined experience levels
        experience_level = None
        for level in experience_levels:
            if level in raw_experience_and_topic:
                experience_level = level
                raw_topic = raw_experience_and_topic.replace(level, '', 1).strip(' -')
                if raw_topic:  # Append only if there's anything left
                    learning_topic = f"{raw_topic} - {learning_topic}"
                break
        
        if not experience_level:
            # If no experience level is matched, use the entire string as the topic
            learning_topic = f"{raw_experience_and_topic} - {learning_topic}"
            experience_level = "Unknown"
    else:
        occupation = experience_level = learning_topic = "Unknown"

    turns = turns_search.group(1).strip() if turns_search else "Unknown"
    use_case = use_case_search.group(1).strip() if use_case_search else "Unknown"
    technical_topic = technical_topic_search.group(1).strip() if technical_topic_search else "Unknown"
    personality = personality_search.group(1).strip() if personality_search else "Unknown"

    return {
        "occupation": occupation,
        "experience_level": experience_level,
        "learning_topic": learning_topic,
        "turns": turns,
        "use_case": use_case,
        "technical_topic": technical_topic,
        "personality": personality,
    }


input_data = '''
**Occupation Topics** - Python Content Creator (Technical Writing/Tutorials) Educational Technologist(python-learning) > Advanced Interview preparation - Proactively ask follow up questions

**Target Number of Turns (User + Assistant)** - 1-5

**Use Case** - Creating a dynamic data visualization dashboard

**Technical Topic** - Interactive charts using Plotly with Dash in Python

**User Personality** - Open-minded, engaged, and elaborate
'''
def format_output(info):
    output = f"""
**User Occupation** - {info["occupation"]}

**User Experience Level** - {info["experience_level"]}

**User Use Case** : {info["learning_topic"]}: {info["use_case"]}

**Technical Topic Suggestion** - {info["technical_topic"]}

**User Personality** - {info["personality"]}

**Target Number of Turns (User + Assistant)** - {info["turns"]}
    """
    return output.strip()

info = extract_info(input_data)
formatted_output = format_output(info)
print(formatted_output)

**User Occupation** - Python Content Creator (Technical Writing/Tutorials) Educational Technologist(python-learning)

**User Experience Level** - Advanced

**User Use Case** : Interview preparation - Proactively ask follow up questions: Creating a dynamic data visualization dashboard

**Technical Topic Suggestion** - Interactive charts using Plotly with Dash in Python

**User Personality** - Open-minded, engaged, and elaborate

**Target Number of Turns (User + Assistant)** - 1-5


In [19]:
import re

def extract_info(data):
    occupation_search = re.search(r'\*\*Occupation Topics\*\* - ([\w\s-]+(?:\(.*?\))?[ \w-]*) > ([\w\s-]+?) - (.+)', data)
    turns_search = re.search(r'\*\*Target Number of Turns \(User \+ Assistant\)\*\* - ([\d-]+)', data)
    use_case_search = re.search(r'\*\*Use Case\*\* - (.+)', data)
    technical_topic_search = re.search(r'\*\*Technical Topic\*\* - (.+)', data)
    personality_search = re.search(r'\*\*User Personality\*\* - (.+)', data)

    experience_levels = ["First Timer", "Beginner", "Intermediate", "Advanced", "Expert"]

    if occupation_search:
        occupation = occupation_search.group(1).strip()
        raw_experience_and_topic = occupation_search.group(2).strip()
        learning_topic = occupation_search.group(3).strip()

        # Match any of the predefined experience levels
        experience_level = None
        for level in experience_levels:
            if level in raw_experience_and_topic:
                experience_level = level
                raw_topic = raw_experience_and_topic.replace(level, '', 1).strip(' -')
                if raw_topic:  # Append only if there's anything left
                    learning_topic = f"{raw_topic} - {learning_topic}"
                break
        
        if not experience_level:
            # If no experience level is matched, use the entire string as the topic
            learning_topic = f"{raw_experience_and_topic} - {learning_topic}"
            experience_level = "Unknown"
    else:
        occupation = experience_level = learning_topic = "Unknown"

    turns = turns_search.group(1).strip() if turns_search else "Unknown"
    use_case = use_case_search.group(1).strip() if use_case_search else "Unknown"
    technical_topic = technical_topic_search.group(1).strip() if technical_topic_search else "Unknown"
    personality = personality_search.group(1).strip() if personality_search else "Unknown"

    return {
        "occupation": occupation,
        "experience_level": experience_level,
        "learning_topic": learning_topic,
        "turns": turns,
        "use_case": use_case,
        "technical_topic": technical_topic,
        "personality": personality,
    }


def clean_and_update_cell_source(cell_source, metadata_dict):
    # We use a set to hold all lines that have been processed and should be removed
    lines_to_remove = {
        "**Occupation Topics** -",
        "**Target Number of Turns (User + Assistant)** -",
        "**Use Case** -",
        "**Technical Topic** -",
        "**User Personality** -"
    }
    
    # Split the cell source into lines and remove unwanted ones
    lines = cell_source.split('\n')
    cleaned_lines = [line for line in lines if not any(line.startswith(prefix) for prefix in lines_to_remove)]
    
    # Update the cell with new metadata
    updated_metadata_lines = [
        f"**User Occupation** - {metadata_dict['occupation']}\n",
        f"**User Experience Level** - {metadata_dict['experience_level']}\n",
        f"**User Use Case** - {metadata_dict['learning_topic']} - {metadata_dict['use_case']}\n",
        f"**Technical Topic Suggestion** - {metadata_dict['technical_topic']}\n",
        f"**User Personality** - {metadata_dict['personality']}\n",
        f"**Target Number of Turns (User + Assistant)** - {metadata_dict['turns']}\n"
    ]
    
    # Join the cleaned lines with the updated metadata, skipping 'Unknown' values
    final_lines = cleaned_lines + [line for line in updated_metadata_lines if "Unknown" not in line]
    return '\n'.join(final_lines).strip()

def modify_markdown_metadata(notebook_path):
    # Load the notebook
    with open(notebook_path, 'r', encoding='utf-8') as f:
        notebook = nbformat.read(f, as_version=4)
    
    # Iterate through each markdown cell and update its content
    for cell in notebook.cells:
        if cell.cell_type == 'markdown':
            metadata_dict = extract_info(cell.source)
            cell.source = clean_and_update_cell_source(cell.source, metadata_dict)
            
    # Remove empty markdown cells after updating
    notebook.cells = [cell for cell in notebook.cells if not (cell.cell_type == 'markdown' and not cell.source.strip())]
    
    # Save the notebook back to the file
    with open(notebook_path, 'w', encoding='utf-8') as f:
        nbformat.write(notebook, f)

def modify_all_notebooks_in_folder(folder_path):
    # Iterate over files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.ipynb'):
            notebook_path = os.path.join(folder_path, filename)
            modify_markdown_metadata(notebook_path)
            print(f'Modified notebook: {filename}')

folder_path = './notebooks/v3/new_pipe/batch_2/'
modify_all_notebooks_in_folder(folder_path)

Modified notebook: Advanced Debugging and Troubleshooting - Debug error trace__0_V3_B.ipynb
Modified notebook: Advanced Debugging and Troubleshooting - Debug error trace__1_V3_B.ipynb
Modified notebook: Advanced Debugging and Troubleshooting - Debug error trace__2_V3_B.ipynb
Modified notebook: Advanced Debugging and Troubleshooting - Debug error trace__3_V3_B.ipynb
Modified notebook: Advanced Debugging and Troubleshooting - Debug error trace__4_V3_B.ipynb
Modified notebook: Advanced Debugging and Troubleshooting - Debug error trace__5_V3_B.ipynb
Modified notebook: Advanced Debugging and Troubleshooting - Debug error trace__6_V3_B.ipynb
Modified notebook: Advanced Debugging and Troubleshooting - Debug error trace__7_V3_B.ipynb
Modified notebook: Advanced Debugging and Troubleshooting - Debug error trace__8_V3_B.ipynb
Modified notebook: Advanced Debugging and Troubleshooting - Debug error trace__9_V3_B.ipynb
Modified notebook: Advanced General coding help - Add comments to code__0_V3_B.i

In [20]:
from tqdm import tqdm
from src.gdrive_api.folder_upload import upload_folder
from src.gdrive_api.auth import build_service

service = build_service('creds/google__sa.json')
# destination_folder_url = "https://drive.google.com/drive/u/0/folders/1Z1bdYMe2Qmo_vs-OaKDaYIiV3rIqLJH9"
# destination_folder_url = "https://drive.google.com/drive/u/2/folders/1sfPFHkXYpKyY41V0pfz3Qw3k4VLy5Hvb"
# destination_folder_url = 'https://drive.google.com/drive/folders/1T14cHENpx7j2aS4Yb_3WhnXhMql0zq2y'
# destination_folder_url = 'https://drive.google.com/drive/u/2/folders/1jV7WA5zB172DJUp7Z2XzHr62E6U6_NtY'
destination_folder_url =  'https://drive.google.com/drive/u/2/folders/16QZtky1kk-yLK19ZEQDy9ErxUMcFxDlU'

uploaded_files = upload_folder(service, 'notebooks/v3/new_pipe/batch_2', destination_folder_url, force_replace = True, is_url=True)

file_path_to_url = {}
with tqdm(total=len(uploaded_files)) as pbar:
    for file_path, file_url in uploaded_files.items():
        if file_url is not None:
            drive_id = file_url.split("id=")[-1].split("&")[0].strip()
            colab_url = f"https://colab.research.google.com/drive/{drive_id}"
            file_path_to_url[file_path] = colab_url
        else:
            print(f"Skipped uploading {file_path}")
        pbar.update(1)

for file_path in file_path_to_url.keys():
    if file_path == "problems.json":
        continue
    if file_path in file_path_to_problem:
        problem = file_path_to_problem[file_path]
        problem["metadata"]["colab_url"] = file_path_to_url[file_path]
        problem["metadata"]["file_path"] = file_path
        problem["metadata"]["batch_idx"] = "batch_2"
    else:
        print(f"Warning: No entry for '{file_path}' found in file_path_to_problem.")

------------------------------------------------------------
Processing directory .: 1 of 0 in total.
Uploading file 1 of 503 in '.', 1 of 503 in total.
Replacing existing file 'Advanced Debugging and Troubleshooting - Debug error trace__0_V3_B.ipynb' with the new version.
File 'Advanced Debugging and Troubleshooting - Debug error trace__0_V3_B.ipynb' has been replaced.
Uploaded 'Advanced Debugging and Troubleshooting - Debug error trace__0_V3_B.ipynb' to folder ID '16QZtky1kk-yLK19ZEQDy9ErxUMcFxDlU'.
Advanced Debugging and Troubleshooting - Debug error trace__0_V3_B.ipynb
Uploading file 2 of 503 in '.', 2 of 503 in total.
Replacing existing file 'Advanced Debugging and Troubleshooting - Debug error trace__1_V3_B.ipynb' with the new version.
File 'Advanced Debugging and Troubleshooting - Debug error trace__1_V3_B.ipynb' has been replaced.
Uploaded 'Advanced Debugging and Troubleshooting - Debug error trace__1_V3_B.ipynb' to folder ID '16QZtky1kk-yLK19ZEQDy9ErxUMcFxDlU'.
Advanced Debugg

100%|██████████| 503/503 [00:00<00:00, 125482.36it/s]

In [21]:
problems

[{'metadata': {'Occupation': 'Product Manager > First Timer Basic scripting - Write simple code in Javascript',
   'colab_url': 'https://colab.research.google.com/drive/1BQkXgGDCGYqZ350joFfoJC1RfTV1lGdt',
   'file_path': 'First Timer Basic scripting - Write simple code in Javascript__0_V3_B.ipynb',
   'batch_idx': 'batch_2'},
  'messages': [{'role': 'user',
    'content': {'use_case': 'Creating a simple interactive dashboard for tracking team performance',
     'technical_topic': 'Integrating JavaScript with HTML/CSS for dynamic content',
     'user_personality': 'Engaged, Open-Minded, and Collaborative'}}]},
 {'metadata': {'Occupation': 'Data Engineer > Intermediate Interview preparation - Debug and fix code',
   'colab_url': 'https://colab.research.google.com/drive/1xcw0b8GzHq5p-OLh9UtacZIiPJeTZoUo',
   'file_path': 'Intermediate Interview preparation - Debug and fix code__0_V3_B.ipynb',
   'batch_idx': 'batch_2'},
  'messages': [{'role': 'user',
    'content': {'use_case': 'Optimizi

In [114]:
len(file_path_to_url)

501

In [17]:
file_path_to_problem

{'First Timer Basic scripting - Write simple code in Javascript__0_V3_B.ipynb': {'metadata': {'Occupation': 'Product Manager > First Timer Basic scripting - Write simple code in Javascript'},
  'messages': [{'role': 'user',
    'content': {'use_case': 'Creating a simple interactive dashboard for tracking team performance',
     'technical_topic': 'Integrating JavaScript with HTML/CSS for dynamic content',
     'user_personality': 'Engaged, Open-Minded, and Collaborative'}}]},
 'Intermediate Interview preparation - Debug and fix code__0_V3_B.ipynb': {'metadata': {'Occupation': 'Data Engineer > Intermediate Interview preparation - Debug and fix code'},
  'messages': [{'role': 'user',
    'content': {'use_case': 'Optimizing data retrieval from a PostgreSQL database',
     'technical_topic': 'SQL query performance tuning',
     'user_personality': 'Analytical, Patient, and Formal'}}]},
 'Advanced General coding help - Explain complex code snippets__0_V3_B.ipynb': {'metadata': {'Occupation'

In [23]:
csv_file = "./notebooks/v3/new_pipe/batch_2/batch_2.csv"
csv_columns = ['colab_url', 'file_path', 'batch_idx']

try:
    # Open the CSV file for writing
    with open(csv_file, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        # Iterate over the list items
        for item in problems:
            metadata = item['metadata']
            # Write a row to the CSV using the metadata
            writer.writerow({column: metadata.get(column) for column in csv_columns})
except IOError:
    print("I/O error")

In [364]:

problems = list(file_path_to_problem.values())

with open('notebooks/v3/new_pipe/v3_1/problems.json', 'w') as f:
    json.dump(sampled_items, f, indent=4)


In [116]:
problems

[{'metadata': {'Occupation': 'Python Developer > First Timer Learning - Learn to use some tool - infra',
   'colab_url': 'https://colab.research.google.com/drive/1Ay4PRjVYrKtWWSAB3fTVBJNbtqQA8sq-',
   'file_path': 'First Timer Learning - Learn to use some tool - infra__0_V3_B.ipynb',
   'batch_idx': 'v3_1'},
  'messages': [{'role': 'user',
    'content': {'use_case': 'Setting up a Python development environment for the first time',
     'technical_topic': 'Virtual environments with virtualenv',
     'user_personality': 'Curious, patient, and independent'}}]},
 {'metadata': {'Occupation': 'Python Developer > First Timer Learning - Learn to use some tool - infra',
   'colab_url': 'https://colab.research.google.com/drive/1GkEbMhX7awWPgn-GGb1D2xBrZTlEI3HQ',
   'file_path': 'First Timer Learning - Learn to use some tool - infra__1_V3_B.ipynb',
   'batch_idx': 'v3_1'},
  'messages': [{'role': 'user',
    'content': {'use_case': 'Writing a basic Python script to automate a simple task',
     

In [256]:
type(file_path_to_url)

dict

In [75]:
df = pd.DataFrame(values_1)
df.to_csv('500.csv')

In [118]:
problems

[{'metadata': {'Occupation': 'Python Developer > First Timer Learning - Learn to use some tool - infra',
   'colab_url': 'https://colab.research.google.com/drive/1Ay4PRjVYrKtWWSAB3fTVBJNbtqQA8sq-',
   'file_path': 'First Timer Learning - Learn to use some tool - infra__0_V3_B.ipynb',
   'batch_idx': 'v3_1'},
  'messages': [{'role': 'user',
    'content': {'use_case': 'Setting up a Python development environment for the first time',
     'technical_topic': 'Virtual environments with virtualenv',
     'user_personality': 'Curious, patient, and independent'}}]},
 {'metadata': {'Occupation': 'Python Developer > First Timer Learning - Learn to use some tool - infra',
   'colab_url': 'https://colab.research.google.com/drive/1GkEbMhX7awWPgn-GGb1D2xBrZTlEI3HQ',
   'file_path': 'First Timer Learning - Learn to use some tool - infra__1_V3_B.ipynb',
   'batch_idx': 'v3_1'},
  'messages': [{'role': 'user',
    'content': {'use_case': 'Writing a basic Python script to automate a simple task',
     

In [119]:
values = []

for problem in problems:
    values.append([
        problem["metadata"]["colab_url"],
        problem["metadata"]["Occupation"],
    ])

In [120]:
values

[['https://colab.research.google.com/drive/1Ay4PRjVYrKtWWSAB3fTVBJNbtqQA8sq-',
  'Python Developer > First Timer Learning - Learn to use some tool - infra'],
 ['https://colab.research.google.com/drive/1GkEbMhX7awWPgn-GGb1D2xBrZTlEI3HQ',
  'Python Developer > First Timer Learning - Learn to use some tool - infra'],
 ['https://colab.research.google.com/drive/1wba2gx-C_sXJt0_Rjrtrj2fSmlbQhOeQ',
  'Python Developer > First Timer Learning - Learn to use some tool - infra'],
 ['https://colab.research.google.com/drive/1Zw9maCm5rvmjPMKhEHY4cP8hcPzMtHRL',
  'Python Developer > First Timer Learning - Learn to program'],
 ['https://colab.research.google.com/drive/11ghRse1yta5q0ecxbZHvUZylpvZ0yUse',
  'Python Developer > First Timer Learning - Learn to program'],
 ['https://colab.research.google.com/drive/1wVBwHp6k5DNj0-rDNLkTlG4sxkF7Nhxd',
  'Python Developer > First Timer Learning - Learn to program'],
 ['https://colab.research.google.com/drive/1OuG92lSdGprumJSplqnheZjAWpAaFJhm',
  'Python Devel

In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Path to your service account key file
SERVICE_ACCOUNT_FILE = 'creds/google__sa.json'

# The scopes required for the Sheets API
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# The ID of your spreadsheet
SPREADSHEET_ID = '1qBU7Kvuuij2fxbqPxebReKMxWgIBmOIE5Gi4ZuX0j_4'

# Authenticate and build the service
creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('sheets', 'v4', credentials=creds)

# Specify the range and values to update
range_ = 'Conversations_Batch_5!A1380:E2000'  # For example, this updates cells from A1 to D5 in Sheet1
values = []

for problem in problems:
    values.append([
        problem["metadata"]["colab_url"],
        problem["metadata"]["Occupation"],
    ])


body = {
    'values': values
}

# Call the Sheets API to update the range
request = service.spreadsheets().values().update(spreadsheetId=SPREADSHEET_ID, range=range_, valueInputOption='RAW', body=body)
response = request.execute()

In [ ]:
values = []
for filename, url in files_and_urls.items():
    # Here, we're assuming the topic is the portion before the first underscore
    topic = filename.split('_')[0]
    values.append([topic, url])

In [137]:
problems

[{'metadata': {'Occupation': 'Mobile App Developer > Expert General coding help - Rewrite code using better formatting / best practises',
   'type': 'query',
   'difficulty': 'Easy',
   'target_length': 1},
  'messages': [{'role': 'user',
    'content': {'use_case': 'Optimizing a function for data retrieval in a mobile app',
     'technical_topic': 'Refactoring code for performance and maintainability',
     'user_personality': 'Detail-oriented and passionate about clean code'}}]},
 {'metadata': {'Occupation': 'Python Developer > Beginner General coding help - Explain complex code snippets',
   'type': 'query',
   'difficulty': 'Easy',
   'target_length': 1},
  'messages': [{'role': 'user',
    'content': {'use_case': 'Parsing JSON data from an API response',
     'technical_topic': 'Working with the `requests` and `json` libraries in Python',
     'user_personality': 'Curious and eager to learn, with a preference for step-by-step explanations'}}]},
 {'metadata': {'Occupation': 'Backen

In [129]:
len(problems)

500